In [1]:
#PreProcessing Pipeline
#Goal of this Notebook is to take all of the athletes in the race dictionary, and process all of their raw data files to make time series files for 1 month and 3 month histories.

In [1]:
import pandas_read_xml as pdx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import re
import os
import glob
import ast

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Garmin preprocessing:

In [3]:
#Returns all the filepaths as a list:
filepaths = (glob.glob('./data/raw/Garmin/*'))

In [7]:
filepaths

['./data/raw/Garmin/02_eric_bayless_updated.csv',
 './data/raw/Garmin/09_Riggs Activities r4 w favorites.csv',
 './data/raw/Garmin/04_BrianGetz_Activities.csv',
 './data/raw/Garmin/03_Keith_Activities_2018-Present.csv',
 './data/raw/Garmin/01_ActivitiesBKK2.csv',
 './data/raw/Garmin/00_Activities_Anel_Akiyanova.csv',
 './data/raw/Garmin/10_Taryn-AllActivities.xlsx',
 './data/raw/Garmin/12_JRM races 8-15-20 and 7-11-20.csv',
 './data/raw/Garmin/08_Nate_Churchwell.csv',
 './data/raw/Garmin/11_v_brad_run_data.numbers',
 './data/raw/Garmin/05_WMRunning.csv',
 './data/raw/Garmin/07_Lars - Garmin.csv',
 './data/raw/Garmin/06_jen-all-activities.csv']

In [13]:
filepaths[0]

'./data/raw/Garmin/02_eric_bayless_updated.csv'

In [20]:
int(filepaths[0][18:][:2])

2

In [8]:
#Read in the athletes.json athlete dictionary:

In [9]:
with open("./data/athletes.json") as json_file: 
    athletes = json.load(json_file)

In [10]:
#Turns a string from a json into its literal evaluation, in this case, a list:
ast.literal_eval(athletes["1"]['race_date_list'])

['2017-01-15', '2016-09-18', '2019-06-02']

In [27]:
len(athletes)

13

In [11]:
#create a list of the races:
list_of_races = [ast.literal_eval(athletes[str(i)]['race_date_list']) for i in range(len(athletes))]

In [12]:
list_of_races

[['2020-05-09'],
 ['2017-01-15', '2016-09-18', '2019-06-02'],
 ['2020-02-22',
  '2019-07-20',
  '2019-04-13',
  '2019-04-13',
  '2019-04-13',
  '2018-11-24',
  '2018-10-07',
  '2018-07-29'],
 ['2018-05-05',
  '2018-06-10',
  '2018-09-02',
  '2019-10-13',
  '2019-07-27',
  '2019-06-09'],
 ['2018-09-15', '2019-05-28', '2019-07-13', '2019-09-14', '2020-02-01'],
 ['2018-10-27',
  '2018-11-22',
  '2018-12-09',
  '2019-03-03',
  '2019-09-01',
  '2019-10-26',
  '2019-12-15',
  '2020-02-09'],
 ['2020-10-05'],
 ['2020-10-17'],
 ['2020-05-09', '2020-05-17', '2020-06-14'],
 ['2016-04-12',
  '2016-07-17',
  '2016-09-25',
  '2016-09-25',
  '2016-12-03',
  '2016-12-17',
  '2017-01-07',
  '2017-01-28',
  '2017-04-09',
  '2017-04-09',
  '2017-04-22',
  '2017-04-22',
  '2017-04-22',
  '2017-11-11',
  '2017-12-02',
  '2017-12-16',
  '2018-03-24',
  '2018-04-21',
  '2019-11-16',
  '2020-03-05',
  '2020-05-10'],
 ['2018-10-27',
  '2018-11-22',
  '2018-12-09',
  '2019-03-03',
  '2019-09-01',
  '2019-10-26'

In [31]:
output_list = []
for i in range(len(list_of_races)):
    races = list_of_races[i]
    athlete = i
    filepath = ''
    for j in range(len(filepaths)):
        if int(filepaths[j][18:][:2]) == i:
            print(f"Matched athlete {i} to filepath {filepaths[j]}")
            filepath = filepaths[j]
    output_list.append((str(i), (filepath, races)))
output_list

Matched athlete 0 to filepath ./data/raw/Garmin/00_Activities_Anel_Akiyanova.csv
Matched athlete 1 to filepath ./data/raw/Garmin/01_ActivitiesBKK2.csv
Matched athlete 2 to filepath ./data/raw/Garmin/02_eric_bayless_updated.csv
Matched athlete 3 to filepath ./data/raw/Garmin/03_Keith_Activities_2018-Present.csv
Matched athlete 4 to filepath ./data/raw/Garmin/04_BrianGetz_Activities.csv
Matched athlete 5 to filepath ./data/raw/Garmin/05_WMRunning.csv
Matched athlete 6 to filepath ./data/raw/Garmin/06_jen-all-activities.csv
Matched athlete 7 to filepath ./data/raw/Garmin/07_Lars - Garmin.csv
Matched athlete 8 to filepath ./data/raw/Garmin/08_Nate_Churchwell.csv
Matched athlete 9 to filepath ./data/raw/Garmin/09_Riggs Activities r4 w favorites.csv
Matched athlete 10 to filepath ./data/raw/Garmin/10_Taryn-AllActivities.xlsx
Matched athlete 11 to filepath ./data/raw/Garmin/11_v_brad_run_data.numbers
Matched athlete 12 to filepath ./data/raw/Garmin/12_JRM races 8-15-20 and 7-11-20.csv


[('0', ('./data/raw/Garmin/00_Activities_Anel_Akiyanova.csv', ['2020-05-09'])),
 ('1',
  ('./data/raw/Garmin/01_ActivitiesBKK2.csv',
   ['2017-01-15', '2016-09-18', '2019-06-02'])),
 ('2',
  ('./data/raw/Garmin/02_eric_bayless_updated.csv',
   ['2020-02-22',
    '2019-07-20',
    '2019-04-13',
    '2019-04-13',
    '2019-04-13',
    '2018-11-24',
    '2018-10-07',
    '2018-07-29'])),
 ('3',
  ('./data/raw/Garmin/03_Keith_Activities_2018-Present.csv',
   ['2018-05-05',
    '2018-06-10',
    '2018-09-02',
    '2019-10-13',
    '2019-07-27',
    '2019-06-09'])),
 ('4',
  ('./data/raw/Garmin/04_BrianGetz_Activities.csv',
   ['2018-09-15', '2019-05-28', '2019-07-13', '2019-09-14', '2020-02-01'])),
 ('5',
  ('./data/raw/Garmin/05_WMRunning.csv',
   ['2018-10-27',
    '2018-11-22',
    '2018-12-09',
    '2019-03-03',
    '2019-09-01',
    '2019-10-26',
    '2019-12-15',
    '2020-02-09'])),
 ('6', ('./data/raw/Garmin/06_jen-all-activities.csv', ['2020-10-05'])),
 ('7', ('./data/raw/Garmin/07

In [21]:
athletes_races_zipped = list(zip(range(len(list_of_races)), list_of_races))

In [22]:
athletes_races_zipped

[(0, ['2020-05-09']),
 (1, ['2017-01-15', '2016-09-18', '2019-06-02']),
 (2,
  ['2020-02-22',
   '2019-07-20',
   '2019-04-13',
   '2019-04-13',
   '2019-04-13',
   '2018-11-24',
   '2018-10-07',
   '2018-07-29']),
 (3,
  ['2018-05-05',
   '2018-06-10',
   '2018-09-02',
   '2019-10-13',
   '2019-07-27',
   '2019-06-09']),
 (4, ['2018-09-15', '2019-05-28', '2019-07-13', '2019-09-14', '2020-02-01']),
 (5,
  ['2018-10-27',
   '2018-11-22',
   '2018-12-09',
   '2019-03-03',
   '2019-09-01',
   '2019-10-26',
   '2019-12-15',
   '2020-02-09']),
 (6, ['2020-10-05']),
 (7, ['2020-10-17']),
 (8, ['2020-05-09', '2020-05-17', '2020-06-14']),
 (9,
  ['2016-04-12',
   '2016-07-17',
   '2016-09-25',
   '2016-09-25',
   '2016-12-03',
   '2016-12-17',
   '2017-01-07',
   '2017-01-28',
   '2017-04-09',
   '2017-04-09',
   '2017-04-22',
   '2017-04-22',
   '2017-04-22',
   '2017-11-11',
   '2017-12-02',
   '2017-12-16',
   '2018-03-24',
   '2018-04-21',
   '2019-11-16',
   '2020-03-05',
   '2020-05-10']

In [24]:
files_races_zipped = list(zip(filepaths, list_of_races))

In [25]:
#Define my inputs:
inputs = list(zip(athletes, files_races_zipped))

In [26]:
inputs

[('0', ('./data/raw/Garmin/02_eric_bayless_updated.csv', ['2020-05-09'])),
 ('1',
  ('./data/raw/Garmin/09_Riggs Activities r4 w favorites.csv',
   ['2017-01-15', '2016-09-18', '2019-06-02'])),
 ('2',
  ('./data/raw/Garmin/04_BrianGetz_Activities.csv',
   ['2020-02-22',
    '2019-07-20',
    '2019-04-13',
    '2019-04-13',
    '2019-04-13',
    '2018-11-24',
    '2018-10-07',
    '2018-07-29'])),
 ('3',
  ('./data/raw/Garmin/03_Keith_Activities_2018-Present.csv',
   ['2018-05-05',
    '2018-06-10',
    '2018-09-02',
    '2019-10-13',
    '2019-07-27',
    '2019-06-09'])),
 ('4',
  ('./data/raw/Garmin/01_ActivitiesBKK2.csv',
   ['2018-09-15', '2019-05-28', '2019-07-13', '2019-09-14', '2020-02-01'])),
 ('5',
  ('./data/raw/Garmin/00_Activities_Anel_Akiyanova.csv',
   ['2018-10-27',
    '2018-11-22',
    '2018-12-09',
    '2019-03-03',
    '2019-09-01',
    '2019-10-26',
    '2019-12-15',
    '2020-02-09'])),
 ('6', ('./data/raw/Garmin/10_Taryn-AllActivities.xlsx', ['2020-10-05'])),
 ('7'

In [31]:
#Testing so I can use this with my below function:
# for athlete, (filepath, race_dates) in inputs:
#     print(athlete)
#     print(filepath)
#     print(race_dates)

In [32]:
def import_transf_garmin2(filepath, athlete_id, race_dates): #Filepath is the csv filepath, athlete_id is a counter variable, race_dates is the list of race dates
    
    #Read in the dataframe:
    df = pd.read_csv(filepath, encoding='latin1')
    #Found out this is very important in garmin files:
    df.replace('--', 0, inplace = True)
    
    #Create athlete_id, file type and race category column:
    df['athlete_id'] = athlete_id
    df['filetype'] = 'garmin'
    df['is_race'] = 0
    
    #Apparently some garmins save different columns:
    if 'Avg Speed' in df.columns:
        print("We have an Avg Speed Column!")
        df['Avg Pace'] = 0
        
        for i in range(len(df)):
            #print(df['Avg Speed'][i])
            try:
                df['Avg Pace'][i] = re.findall("\d?\d:\d\d", df['Avg Speed'][i])[0]
                #print(df['Avg_Pace'][i])
            except:
                df['Avg Pace'][i] = '00:00:00'
        #Best we can do for now is set the best pace to the average pace:
        df['Best Pace'] = df['Avg Pace']
    
    
    #Define the columns to have their data types transformed:
    obj_to_num_cols = ['Calories', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence', 'Max Run Cadence', 'Elev Gain', 'Elev Loss', 'Avg Run Cadence.1', 'Avg Power', 'Max Power']
    pace_cols = ['Avg Pace', 'Best Pace']
    
    #Convert the numeric columns:
    for col in obj_to_num_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            try:
                df[col] = pd.to_numeric(df[col].str.extract(r'd+'))
            except:
                pass
    
    #Convert the pace columns:
    for col in pace_cols:
        for i in range(len(df[col])):
            try:
                df[col][i] = pd.to_timedelta('00:'+df[col][i])
            except:
                #print(f'col={col}')
                #print(f'i= {i}')
                #print(f'df[col][i] = {df[col][i]}')
                if df[col][i]==0:
                    df[col][i] = pd.to_timedelta('00:00:00')
                else:
                    pass
                pass
    
    #Clean up the column names:
    #This is to remove the (R) logo so it doesn't cause any issues later:
    r_logo = df.columns[df.columns.str.contains('Training Stress')][0][-1]
    #Clean up the column names:
    df.columns = df.columns.str.replace('/', '').str.lower().str.replace('.', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(r_logo, '')
    df.set_index(df['date'], inplace = True)
    df.index = pd.to_datetime(df.index)
    ###Write a function to change the race dates in race_dates to a '1'
    
    #Write out the file to the data folder:
    df.to_csv(f'./data/clean/garmin_clean/clean_{athlete_id}.csv')

In [33]:
for athlete, (filepath, race_dates) in output_list:
    print(athlete)
    try:
        import_transf_garmin2(filepath, athlete, race_dates)
    except:
        print(f'Athlete {athlete} was not able to be imported.')

0
1
2
We have an Avg Speed Column!
3
4
5
6
7
8
9
We have an Avg Speed Column!
10
Athlete 10 was not able to be imported.
11
Athlete 11 was not able to be imported.
12


In [34]:
#Didn't get 10 or 11. 

In [36]:
import_transf_garmin2('./data/raw/Garmin/10_Taryn-AllActivities.csv', 10, ['2018-10-27',
    '2018-11-22',
    '2018-12-09',
    '2019-03-03',
    '2019-09-01',
    '2019-10-26',
    '2019-12-15',
    '2020-02-09'])

We have an Avg Speed Column!


In [37]:
import_transf_garmin2('./data/raw/Garmin/11_v_brad_run_data.csv', 11, ['2021-01-17'])

In [49]:
#The  below function chops up the data and puts it into folders according to athlete #

In [38]:
def set_up_for_prediction(df_filepath, athlete_dict):
    
    '''
    Inputs: csv filepath, and athlete dictionary
    -Dictionary contains an entry for race date, keyed to athlete id
    Outputs: 
        1) a csv with races indicated, indexed by time series
        2) a csv with summary values for 1 week, 2 weeks, 1 month, 2 months, 3 months
            - If values are not available, they will be 0s
    '''
    #Read in the csv:
    working = pd.read_csv(df_filepath, index_col='date').drop('date.1', axis =1)
    
    athlete_id = working['athlete_id'][0]
    #print(athlete_id)
    
    #get the race dates for the selected athlete: (note that the json file means I need to reference a string for the id)
    race_dates = ast.literal_eval(athlete_dict[f"{athlete_id}"]['race_date_list'])
    #print(race_dates)
    #print(type(race_dates))
    
    #Set the index as a datetime:
    working.index = pd.to_datetime(working.index)
    
    #Create a month interval: (we can multiply by 3 to get the 3 month values)
    month = timedelta(31)
    day = timedelta(1)
    
    #Create a time of day column:
    time_of_day = [re.findall("\d\d:\d\d:\d\d", working.index.to_series().astype('str')[i])[0] for i in range(len(working))]
    
    #Drop the hours and minutes and seconds:
    working.index = pd.DatetimeIndex(working.index).normalize()
    
    #Label all of the races:
    working['is_race'] = np.where(working.index.isin(race_dates), 1, 0)
    
    #Create a folder for the athlete to keep things tidy:
    parent_path = './data/races'
    directory = f'athlete_{athlete_id}'
    path = os.path.join(parent_path, directory)
    os.mkdir(path)
    
    
    #select the last month and last 3 months of data from the working file, and export them as separate csvs to the 'races' folder:
    
    for i in range(len(race_dates)):
        #i is the counter, find the 1 month and 3 month data:
        rd = pd.to_datetime(race_dates[i])
        race_1_mo_back = working[rd+day:rd-month]
        race_1_mo_back.sort_index(inplace=True)
        race_1_mo_back.to_csv(f'{path}/athlete_{athlete_id}_race_{i}_1_mo.csv')
        try:
            #This way if there is not 3 months data, it just skips out.
            race_3_mo_back = working[rd+day:(rd-month*3)]
            race_3_mo_back.sort_index(inplace=True)
            race_3_mo_back.to_csv(f'{path}/athlete_{athlete_id}_race_{i}_3_mo.csv')
        except:
            pass


In [39]:
clean_filepaths = (glob.glob('./data/clean/garmin_clean/*'))

In [40]:
clean_filepaths

['./data/clean/garmin_clean/clean_6.csv',
 './data/clean/garmin_clean/clean_7.csv',
 './data/clean/garmin_clean/clean_5.csv',
 './data/clean/garmin_clean/clean_4.csv',
 './data/clean/garmin_clean/clean_0.csv',
 './data/clean/garmin_clean/clean_1.csv',
 './data/clean/garmin_clean/clean_3.csv',
 './data/clean/garmin_clean/clean_2.csv',
 './data/clean/garmin_clean/clean_12.csv',
 './data/clean/garmin_clean/clean_11.csv',
 './data/clean/garmin_clean/clean_10.csv',
 './data/clean/garmin_clean/clean_9.csv',
 './data/clean/garmin_clean/clean_8.csv']

In [41]:
for filepath in clean_filepaths:
    set_up_for_prediction(filepath, athletes)

### Strava preprocessing: